 # Clean data

In [25]:
import pandas as pd
import re
from pprint import pprint
SOURCE_DATA_FILE = "Numbers to parsing.tsv"

In [26]:
source_table = pd.read_csv(SOURCE_DATA_FILE, sep='\t')

In [27]:
source_table.head(5)

,query,alternative query,main passage,citation
0,踢猫效应在直面上如何体现？,踢猫效应最初指的是什么？,一位父亲的同事耽误了他很多 时间，影响了他当天工作的完成。想 着明天要挨上司的责骂，他一肚子...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
1,解释一下踢猫效应,NaN,个体会对 弱于自己或者等级低于自己的对象 发泄不满情绪，而产生坏情绪传染 的连锁反应。但是这...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
2,踢猫效应会带来什么连锁反应？,NaN,假如这只猫对我们不重要，而 且它选择了反抗，我们很有可能和 它陷入战争。人踢了猫一下，猫也会...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
3,根据踢猫效应，踢猫会带来什么后果,踢猫又可能带来的结果有哪些？,假如这只猫对我们不重要，而 且它选择了隐忍，这是最好的情况， 它默默地承受了你的坏心情，没有...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"
4,不踢猫的话，可以更好地发泄负面情绪吗？,有在被人身上发泄情绪的冲动时候可以如何更好的解决问题？,当然是可以的，但是并非简单 粗暴地把猫一脚踢开。我们的家人、 朋友、同事、同学都可以成为这只...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42. \n"


In [72]:
# define some stuff for storing queries, passages, and relevance marking
queries = {} # dictionary of 'qid': 'query', ...
passages = {} # dictionary of 'pid': 'passage', ...
citations = {} # dictionary of 'pid': 'citation', ...
matches = [] # dictionary of matching passages entries ('qid', 'pid)
es_df = pd.DataFrame() # this is the table for elastic search index

# numbering counters
qid_counter = 1
pid_counter = 1

for row in source_table.iterrows():

    row_index = row[0]
    row_content = row[1]
    query = str(row_content['query'])
    alt_query = str(row_content['alternative query'])
    passage = str(row_content['main passage'])
    citation = str(row_content['citation'])


    # trim unnecessary spaces and tabs
    query = query.strip()
    alt_query = alt_query.strip()
    passage = passage.strip()
    citation = citation.strip()

    # trim [] from citation
    citation = re.sub(r'\[\d\]', '', citation)

    # strip space in chinese
    passage = re.sub(r' ', '', passage)
    # fix unwanted new lines in chinese
    passage = re.sub(r'(?<![。？！])\n', '', passage)
    # strip numbering at passage beginning
    passage = re.sub(r'^(\d\.)', '', passage)

    # store the entry in dic and record match
    qid = 'q' + str(qid_counter)
    pid = 'p' + str(pid_counter)
    queries[qid] = query
    passages[pid] = passage
    citations[pid] = citation
    matches.append({'qid': qid, 'pid': pid})
    qid_counter += 1
    pid_counter += 1

    if alt_query != "nan":
        # there is an alt query, make extra query
        qid = 'q' + str(qid_counter)
        queries[qid] = alt_query
        matches.append({'qid': qid, 'pid': pid})
        qid_counter += 1
        es_df = es_df.append({'qid': 'q' + str(qid_counter - 2), 'pid': pid, 'query': query, 'passage': passage, 'alt_query': alt_query, 'alt_qid': qid, 'citation': citation}, ignore_index=True)
    else:
        alt_query = None
        es_df = es_df.append({'qid': qid, 'pid': pid, 'query': query, 'passage': passage, 'alt_query': None, 'alt_qid': None, 'citation': citation}, ignore_index=True)

In [73]:
es_df = es_df[['qid', 'query', 'alt_qid', 'alt_query', 'pid', 'passage', 'citation']]

In [74]:
es_df.sample(30)

,qid,query,alt_qid,alt_query,pid,passage,citation
37,q54,初中生的自尊有什么特点,None,None,p38,初中生自尊发展的特点研究。初中生自尊总的特点是不稳定的（杨丽珠，1996；黄煌峰、莫雷，19...,"魏美丹. 初中生亲子沟通与生活满意度的关系[D].福建师范大学,2019."
130,q203,当孩子已经有许多压力时，父母的指责是否会对孩子造成二次伤害？,q204,父母的言语指责会给孩子带来怎样的伤害？,p131,坐在沙发上后，她开始向我哭诉来自学业上的压力，和同学之间的矛盾，以及因学习成绩不理想而受到的...,李曉娟. 非暴力溝通：架起親子溝通“彩虹橋”[N]. 健康報. 2021-06-01 (006).
124,q191,暴力是否能解决亲子矛盾的问题？,q192,亲子矛盾能使用暴力去解决吗？,p125,“非暴力沟通”是美国威斯康星大学临床心理学博士、国际非暴力沟通中心创始人马歇尔·卢森堡博士发...,李曉娟. 非暴力溝通：架起親子溝通“彩虹橋”[N]. 健康報. 2021-06-01 (006).
236,q385,明确的亲子界限是怎么样的？,None,None,p237,她大约在3年级的时候便开始一个人睡（书房），初中一年级时有了完全属于自己的房间，父母进到房间...,"葛虹宇. 高中生亲子心理边界的调查研究[D].哈尔滨师范大学,2028"
110,q171,代沟为什么会存在,None,None,p111,"在任何社会中,新老两代人由于年龄、经历、历史条件、看问题的角度等有所不同,都必然会在心理活动...","胡悦. 亲子沟通与青少年健康成长[D].哈尔滨工程大学,2007."
131,q205,大学生中，亲子关系之间矛盾的发生率是多少？,q206,在大学生中，亲子间发生矛盾的可能性是多少？,p132,结果显示，公共卫生事件期间大学生或多或少都会与父母发生矛盾，其中，偶尔发生矛盾占46.13%...,"大学生亲子关系调查与对策分析[J]. 张若冰,曹海腾,李雪萌,胡志强,叶建亚. 家庭生活指..."
49,q74,为什么学生会有悲观心理（防御性悲观）,q75,亲子沟通和防御性悲观有什么关系,p50,Martin通过访谈，于2003年开展了一项关于大学生为何会采用防御性悲观策略的质性研究，结...,"文碧麟. 初中生亲子沟通与考试焦虑的关系[D].四川师范大学,2021."
207,q353,用专横的态度和子女沟通为什么没有效果？,None,None,p208,的父母对子女具有明显的心理优势，认为沟通就是为了让子女听话，习惯用自己的经验开展教导、用控制...,"马明洋.亲子沟通的技巧[J].心理与健康,2020(06):70-71."
3,q5,根据踢猫效应，踢猫会带来什么后果,q6,踢猫又可能带来的结果有哪些？,p4,假如这只猫对我们不重要，而且它选择了隐忍，这是最好的情况，它默默地承受了你的坏心情，没有把它...,"李秋生,王小文. 踢猫效应与情绪传递[J]. 大众心理学,2016(10):41-42."
83,q127,性别如何影响青少年的自我控制力,None,None,p84,在性别方面，自我控制总分和思维自我控制维度在性别上具有显著性差异，女生自我控制总分和思维自我...,"姬莉. 青少年亲子沟通、自我控制与冒险行为的关系及教育对策[D].河南大学,2020."


In [75]:
queries["q13"]

'亲子关系会影响孩子适应社会的能力吗'

In [76]:
citations["p111"]

'胡悦. 亲子沟通与青少年健康成长[D].哈尔滨工程大学,2007.'

In [77]:
len(es_df)

267

In [81]:
len(passages)

267

# Dunk into elastic search

In [82]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch()

In [83]:
# dump into elasticsearch
temp_bulk_actions = []
for index, row in es_df.iterrows():
	action = {
		"_index": "ctb-nlp-v1",
		"_source": {
			'qid': row['qid'], 
			'query': row['query'], 
			'alt_qid': row['alt_qid'], 
			'alt_query': row['alt_query'], 
			'pid': row['pid'], 
			'passage': row['passage'], 
			'citation': row['citation'],
			}
		}
	temp_bulk_actions.append(action)

helpers.bulk(es, temp_bulk_actions)

/Users/leonlu-m1/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


(267, [])

In [84]:
# don't know what this does
es.indices.refresh(index="efaqa-70")

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

# Now make training data

The